In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer
%load_ext rich

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [10]:
raw_dataset  = load_dataset("glue", "mrpc")
raw_dataset


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [11]:
raw_train  = raw_dataset["train"]
raw_train[0]


{
    'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
    'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
    'label': 1,
    'idx': 0
}

In [12]:
raw_train.features


{
    'sentence1': Value('string'),
    'sentence2': Value('string'),
    'label': ClassLabel(names=['not_equivalent', 'equivalent']),
    'idx': Value('int32')
}

In [14]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tk1 = tokenizer(raw_train[0]["sentence1"])
tk2 = tokenizer(raw_train[0]["sentence2"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs


{
    'input_ids': [
        101,
        2023,
        2003,
        1996,
        2034,
        6251,
        1012,
        102,
        2023,
        2003,
        1996,
        2117,
        2028,
        1012,
        102
    ],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

In [18]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])


[
    '[CLS]',
    'this',
    'is',
    'the',
    'first',
    'sentence',
    '.',
    '[SEP]',
    'this',
    'is',
    'the',
    'second',
    'one',
    '.',
    '[SEP]'
]